# Data Loading

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib notebook
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
import missingno as msno
from datetime import datetime as dt


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('C:\\kerja\\NIA\\amy\\master_um\\data_analytics_WQD7003\\group_project\\project'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

C:\kerja\NIA\amy\master_um\data_analytics_WQD7003\group_project\project\items.csv
C:\kerja\NIA\amy\master_um\data_analytics_WQD7003\group_project\project\item_categories.csv
C:\kerja\NIA\amy\master_um\data_analytics_WQD7003\group_project\project\sales_train.csv
C:\kerja\NIA\amy\master_um\data_analytics_WQD7003\group_project\project\sample_submission.csv
C:\kerja\NIA\amy\master_um\data_analytics_WQD7003\group_project\project\shops.csv
C:\kerja\NIA\amy\master_um\data_analytics_WQD7003\group_project\project\test.csv


In [2]:
train2 = pd.read_csv('C:\\kerja\\NIA\\amy\\master_um\\data_analytics_WQD7003\\group_project\\project\\sales_train.csv', index_col='date', parse_dates= True)
train = pd.read_csv('C:\\kerja\\NIA\\amy\\master_um\\data_analytics_WQD7003\\group_project\\project\\sales_train.csv')
sample_submission = pd.read_csv('C:\\kerja\\NIA\\amy\\master_um\\data_analytics_WQD7003\\group_project\\project\\sample_submission.csv')
items = pd.read_csv('C:\\kerja\\NIA\\amy\\master_um\\data_analytics_WQD7003\\group_project\\project\\items.csv')
item_categories = pd.read_csv('C:\\kerja\\NIA\\amy\\master_um\\data_analytics_WQD7003\\group_project\\project\\item_categories.csv')
shops = pd.read_csv('C:\\kerja\\NIA\\amy\\master_um\\data_analytics_WQD7003\\group_project\\project\\shops.csv')
test = pd.read_csv('C:\\kerja\\NIA\\amy\\master_um\\data_analytics_WQD7003\\group_project\\project\\test.csv')

# Data Cleaning & Data Visualization

In [32]:
def eda(data):
    print("----------Top-5- Record----------")
    print(data.head(5))
    print("-----------Information-----------")
    print(data.info())
    print("-----------Data Types-----------")
    print(data.dtypes)
    print("----------Missing value-----------")
    print(data.isnull().sum())
    print("----------Null value-----------")
    print(data.isna().sum())
    print("----------Shape of Data----------")
    print(data.shape)

def graph_insight(data):
    print(set(data.dtypes.tolist()))
    df_num = data.select_dtypes(include = ['float64', 'int64'])
    df_num.hist(figsize=(16, 16), bins=50, xlabelsize=8, ylabelsize=8);
    
def drop_duplicate(data, subset):
    print('Before drop shape:', data.shape)
    before = data.shape[0]
    data.drop_duplicates(subset,keep='first', inplace=True) #subset is list where you have to put all column for duplicate check
    data.reset_index(drop=True, inplace=True)
    print('After drop shape:', data.shape)
    after = data.shape[0]
    print('Total Duplicate:', before-after)

In [22]:
#train.tail()
#sample_submission.tail()
train.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


In [37]:
#there are 2935849 rows of data with 4 input variables and a single target variable.
#The dataset is split into train and test sets with 1967018 rows used for model training and 968831 for model evaluation.
data = train2.values
# split into input and output elements
X, y = data[:, :-1], data[:, -1]
# summarize the shape of the dataset
print(X.shape, y.shape)
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# summarize the shape of the train and test sets
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2935849, 4) (2935849,)
(1967018, 4) (968831, 4) (1967018,) (968831,)


In [40]:
#this model identified and removed 196,702 outliers and achieved a MAE of about 0.455

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import IsolationForest
from sklearn.metrics import mean_absolute_error
data = train2.values
# split into input and output elements
X, y = data[:, :-1], data[:, -1]
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# summarize the shape of the training dataset
print(X_train.shape, y_train.shape)
# identify outliers in the training dataset
iso = IsolationForest(contamination=0.1)
yhat = iso.fit_predict(X_train)
# select all rows that are not outliers
mask = yhat != -1
X_train, y_train = X_train[mask, :], y_train[mask]
# summarize the shape of the updated training dataset
print(X_train.shape, y_train.shape)
# fit the model
model = LinearRegression()
model.fit(X_train, y_train)
# evaluate the model
yhat = model.predict(X_test)
# evaluate predictions
mae = mean_absolute_error(y_test, yhat)
print('MAE: %.3f' % mae)

(1967018, 4) (1967018,)
(1770316, 4) (1770316,)
MAE: 0.455


In [47]:
#this model identified and removed 141,835 outliers and achieved a MAE of about 0.434

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import mean_absolute_error
data = train2.values
# split into input and output elements
X, y = data[:, :-1], data[:, -1]
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# summarize the shape of the training dataset
print(X_train.shape, y_train.shape)
# identify outliers in the training dataset
lof = LocalOutlierFactor()
yhat = lof.fit_predict(X_train)
# select all rows that are not outliers
mask = yhat != -1
X_train, y_train = X_train[mask, :], y_train[mask]
# summarize the shape of the updated training dataset
print(X_train.shape, y_train.shape)
# fit the model
model = LinearRegression()
model.fit(X_train, y_train)
# evaluate the model
yhat = model.predict(X_test)
# evaluate predictions
mae = mean_absolute_error(y_test, yhat)
print('MAE: %.3f' % mae)

(1967018, 4) (1967018,)
(1825183, 4) (1825183,)
MAE: 0.434


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import OneClassSVM
from sklearn.metrics import mean_absolute_error
data = train2.values
# split into input and output elements
X, y = data[:, :-1], data[:, -1]
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# summarize the shape of the training dataset
print(X_train.shape, y_train.shape)
# identify outliers in the training dataset
ee = OneClassSVM(nu=0.01)
yhat = ee.fit_predict(X_train)
# select all rows that are not outliers
mask = yhat != -1
X_train, y_train = X_train[mask, :], y_train[mask]
# summarize the shape of the updated training dataset
print(X_train.shape, y_train.shape)
# fit the model
model = LinearRegression()
model.fit(X_train, y_train)
# evaluate the model
yhat = model.predict(X_test)
# evaluate predictions
mae = mean_absolute_error(y_test, yhat)
print('MAE: %.3f' % mae)

(1967018, 4) (1967018,)
